In [47]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
D_2017_1_01 = 'https://raw.githubusercontent.com/Amcdonne95/SeniorDesignData/master/Drive%20Auto%20Receivables%20Trust/DRIVE%202017-1/2017/05%20May/2017.06.15_Drive%20Auto%20Receivables%20Trust%202017-1%20Data%20Tape.csv'

In [52]:
df = pd.read_csv(D_2017_1_01, memory_map=True, low_memory=False)

In [53]:
groupDict = {
    'vehicleNewUsedCode':{
        1:'New',
        2:'Used'
    },
    'vehicleTypeCode':{
        1:'Car',
        2:'Truck',
        3:'SUV',
        4:'Motorcycle',
        98:'Other',
        99:'Unknown'
    },
    'subvented':{
        0:'Not Subvented',
        1:'Rate Subvented',
        2:'Cash Subvented',
        3:'Rate & Cash Subvented',
        98:'Other Subvention'
    },
    'obligorIncomeVerificationLevelCode':{
    1:'Not Stated, Not Verified',
    2:'Not Verified',
    3:'Verified',
    4:'Verified: 12 M',
    5:'Verified: 24 M'
    },
    'obligorEmploymentVerificationCode':{
    1:'Not Stated, Not Verified',
    2:'Not Verified',
    3:'Third Party Verified'
    }
    #'vehicleValueAmount':
}

In [54]:
def mapData(deal, mapDict, col_list):
    for i in col_list:
        # deal[str(i)] = pd.to_numeric(deal[str(i)], errors='coerce')
        deal[str(i)] = deal[str(i)].map(mapDict[str(i)])
    return deal


def calculate_metrics(data):
    WA = lambda x: np.average(x, axis=0, weights=data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])*100
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount'] / (data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount'] / data['vehicleValueAmount'])*100
    return data

def static_table(data, mappingDict):
    WA = lambda x: np.average(x, axis=0, weights=data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    metric = ['vehicleNewUsedCode', 'vehicleManufacturerName', 'vehicleModelName', 'vehicleModelYear',
              'vehicleTypeCode', 'obligorGeographicLocation',
              'obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode', 'subvented',
              'originalLoanTerm', 'remainingTermToMaturityNumber', 'originalInterestRatePercentage', 'LTV']
    calculate_metrics(data)
    dfStats = pd.DataFrame()
    WADict = {}

    for i in metric:
        # Cleaning Steps
        if (data[i].dtypes == 'object') and (i in ['vehicleNewUsedCode', 'vehicleModelYear', 'vehicleTypeCode',
                                                   'obligorIncomeVerificationLevelCode',
                                                   'obligorEmploymentVerificationCode', 'subvented']):
            data[i] = pd.to_numeric(data[i], errors='coerce')

            if i == 'subvented':
                data[i] = data[i].fillna(3)
                mapData(data, mappingDict, ['subvented'])

        if i == 'vehicleNewUsedCode':
            mapData(data, mappingDict, ['vehicleNewUsedCode'])
            stats = data[['pctDeal']].groupby(data[i]).sum()*100  # .reset_index()
            stats = stats.round(2)
            yearRange = [str(data['vehicleModelYear'].min()) + '-' + str(data['vehicleModelYear'].max())]
            yearRange = pd.DataFrame(yearRange, index=['Model Year Range'], columns=['pctDeal'])  # .reset_index()
            stats = stats.append(yearRange)
            dfStats = dfStats.append(stats)
            continue

        if i in ['originalLoanTerm', 'remainingTermToMaturityNumber', 'originalInterestRatePercentage', 'LTV']:
            
            #if i == 'originalLoanTerm' or 'remainingTermToMaturityNumber':
             #   WADict[i] = WA(data[i])
              #  continue
            WADict[i] = WA(data[i]) #need to rename these variables
            if i == 'originalInterestRatePercentage':
                WADict[i] = WA(data[i])*100
            if i == 'LTV':
                dfTemp = pd.DataFrame.from_dict(WADict, orient='index').rename(index={'originalLoanTerm':'WA Original Term', 'remainingTermToMaturityNumber':'WA Remaining Term', 'originalInterestRatePercentage':'WA APR','LTV':'WA LTV'}, columns={0: 'pctDeal'}).round(2)
                # print(dfTemp)
                dfStats = dfStats.append(dfTemp)
            continue

        if i == 'vehicleTypeCode':
            mapData(data, mappingDict, ['vehicleTypeCode'])
        if i in ['obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode']:
            mapData(data, mappingDict, [i])

        stats = (data[['pctDeal']].groupby(data[i]).sum()).sort_values(['pctDeal'], ascending=False).head(3)*100
        stats = stats.round(2)

        if len(stats.index) != 3:
            for i in range(0, (3 - len(stats.index))):
                paddedRow = pd.DataFrame(['--'], index=['--'], columns=['pctDeal'])
                stats = stats.append(paddedRow)

        dfStats = dfStats.append(stats)

    return dfStats.reset_index()

In [55]:
table_data = static_table(df, groupDict)

In [56]:
#table_data.astype(str)
table_data

,index,pctDeal
0,New,24.09
1,Used,75.91
2,Model Year Range,2002-2018
3,NISSAN,13.73
4,TOYOTA,8.44
5,CHEVROLET,7.94
6,Altima,4.26
7,Ram 1500,3.06
8,Sentra,2.97
9,2017,18.95


In [57]:
metricLis = ['Vehicle Stats', 'Vehicle Stats', 'Vehicle Stats',
            'Make', 'Make', 'Make',
            'Model', 'Model', 'Model',
            'Year', 'Year', 'Year',
            'Type', 'Type', 'Type',
            'State', 'State', 'State',
            'Income Verification', 'Income Verification', 'Income Verification',
            'Employment Verification', 'Employment Verification', 'Employment Verification',
            'Subvented', 'Subvented', 'Subvented',
            'Weighted Average Metrics', 'Weighted Average Metrics', 'Weighted Average Metrics',
            'Weighted Average Metrics']

In [58]:
table_data['metric'] = metricLis

In [59]:
table_data

,index,pctDeal,metric
0,New,24.09,Vehicle Stats
1,Used,75.91,Vehicle Stats
2,Model Year Range,2002-2018,Vehicle Stats
3,NISSAN,13.73,Make
4,TOYOTA,8.44,Make
5,CHEVROLET,7.94,Make
6,Altima,4.26,Model
7,Ram 1500,3.06,Model
8,Sentra,2.97,Model
9,2017,18.95,Year


In [60]:
 table = pd.pivot_table(table_data, values='pctDeal', index='index', columns=['metric'])

DataError: No numeric types to aggregate

In [38]:
finaldf = pd.DataFrame(columns=list(range(0,12)))
finaldf.iloc[:][0] = table_data['index'].iloc[0:3]
finaldf.iloc[:][1] = table_data['pctDeal'].iloc[0:3]
finaldf.iloc[:][2] = table_data['index'].iloc[3:6]
finaldf.iloc[:][3] = table_data['pctDeal'].iloc[3:6]
finaldf.iloc[:][4] = table_data['index'].iloc[6:9]
finaldf.iloc[:][5] = table_data['pctDeal'].iloc[6:9]
finaldf.iloc[:][6] = table_data['index'].iloc[9:12]
finaldf.iloc[:][7] = table_data['pctDeal'].iloc[9:12]
finaldf.iloc[:][8] = table_data['index'].iloc[15:18]
finaldf.iloc[:][8] = table_data['pctDeal'].iloc[15:18]

In [40]:
transfer_dict = {
    0: [table_data['index'].iloc[0:3]],
    1: table_data['pctDeal'].iloc[0:3]
}

In [39]:
finaldf

,0,1,2,3,4,5,6,7,8,9,10,11
0,New,24.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Used,75.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Model Year Range,2002-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
print(list(range(0, 12, 2)))
lis = list(range(0, 12))

[0, 2, 4, 6, 8, 10]


In [27]:
print(lis)
print(lis[::2])
print(lis[1::2])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 2, 4, 6, 8, 10]
[1, 3, 5, 7, 9, 11]


In [28]:
def arrange_df(data, col_len):
    df = pd.DataFrame(columns=list(range(0,col_len)))
    row_start = 0
    row_end = 3
    lis = list(range(0, 11))
    for i in lis[::2]:
        df.iloc[:][i] = data['index'].iloc[row_start:row_end]
        row_start += 3
        row_end += 3
    row_start = 0
    row_end = 3
    for j in lis[1::2]:
        df.iloc[:][j] = data['pctDeal'].iloc[row_start:row_end]
        row_start += 3
        row_end += 3
    return df

In [29]:
last_df = arrange_df(table_data, 12)

In [30]:
last_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,New,24.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Used,75.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Model Year Range,2002-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
finaldf

,0,1,2,3,4,5,6,7,8,9,10,11
0,New,24.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Used,75.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Model Year Range,2002-2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
table_data['index'].iloc[0:3]

0                 new
1                used
2    Model Year Range
Name: index, dtype: object

In [ ]:
trace = go.Table(
    header=dict(values=df.columns,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df.Rank, df.State, df.Postal, df.Population],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.iplot(data, filename = 'pandas_table')

In [18]:
trace1 = go.Table(
    header=dict(values=['Vehicle Stats','Percent of Deal']),
    cells=dict(values=[[table_data['index'].iloc[0:3]], [table_data['pctDeal'].iloc[0:3]]])
)
data = [trace1] 
py.iplot(data, filename = 'basic_table')

In [139]:
headerColor = 'grey'
evenColor = 'lightgrey'
oddColor = 'white'
trace = go.Table(
    type='table',
    columnwidth=[300],
    header=dict(
        values=['Vehicle Stats','Deal (%)',
                'Make', 'Deal (%)',
                'Model', 'Deal (%)',
                'Model Year', 'Deal (%)',
                'Vehicle Type', 'Deal (%)',
                "Obligor's State", 'Deal (%)'
               ],
        align='center',
        font = dict(color = 'black', size = 14),
        line=dict(color='black'),
        fill=dict(color=headerColor),
        height=[60]
    ),
    cells=dict(
        values=[[table_data['index'].iloc[0:3]], [table_data['pctDeal'].iloc[0:3]],
                [table_data['index'].iloc[3:6]], [table_data['pctDeal'].iloc[3:6]],
                [table_data['index'].iloc[6:9]], [table_data['pctDeal'].iloc[6:9]],
                [table_data['index'].iloc[9:12]], [table_data['pctDeal'].iloc[9:12]],
                [table_data['index'].iloc[12:15]], [table_data['pctDeal'].iloc[12:15]],
                [table_data['index'].iloc[15:18]], [table_data['pctDeal'].iloc[15:18]]
               ],
        line=dict(color='black'),
        fill=dict(color=[[oddColor, evenColor, oddColor]])
    )
)
data = [trace] 
py.iplot(data, filename = 'basic_table')

In [138]:
headerColor = 'grey'
evenColor = 'lightgrey'
oddColor = 'white'
trace2 = go.Table(
    type='table',
    columnwidth=[300],
    header=dict(
        values=['Income Verification','Deal (%)',
                'Employment Verification', 'Deal (%)',
                'Subvented', 'Deal (%)',
                'Weighted Average', 'Metrics',
                
               ],
        align='center',
        font = dict(color = 'black', size = 14),
        line=dict(color='black'),
        fill=dict(color=headerColor),
        height=[60]
    ),
    cells=dict(
        values=[[table_data['index'].iloc[18:21]], [table_data['pctDeal'].iloc[18:21]],
                [table_data['index'].iloc[21:24]], [table_data['pctDeal'].iloc[21:24]],
                [table_data['index'].iloc[24:27]], [table_data['pctDeal'].iloc[24:27]],
                [table_data['index'].iloc[27:30]], [table_data['pctDeal'].iloc[27:30]],
                
               ],
        line=dict(color='black'),
        fill=dict(color=[[oddColor, evenColor, oddColor]])
    )
)
data = [trace2]
py.iplot(data, filename = 'table2')

In [110]:
print(table_data['index'].iloc[18])
print(table_data['pctDeal'].iloc[18])

Not Verified
84.1


In [96]:
table_data

,index,pctDeal
0,New,24.09
1,Used,75.91
2,Model Year Range,2002-2018
3,NISSAN,13.73
4,TOYOTA,8.44
5,CHEVROLET,7.94
6,Altima,4.26
7,Ram 1500,3.06
8,Sentra,2.97
9,2017,18.95
